# Module 2: Scaling Agentic Tooling Using AgentCore Gateway

In previous lab, you built a Sports Agent locally with tools that worked great for development and testing. But what happens when you want to scale this solution for production? What if other teams want to build agents that also need access to match information and player data? These are thing you need to consider scaling through the cloud, so you can sharing tools across multiple agents, teams, and applications requires a different approach. 

![scale-tool-gateway.png](../static/scale-tool-gateway.png)

In this Lab, you will scale your agent tools for production using Amazon Bedrock AgentCore Gateway. Instead of embedding tool logic directly in your agent code, you'll move the tools into Lambda functions and register them as gateway targets. AgentCore Gateway then transforms these Lambda functions into a centralized MCP-compatible tools that your agents can access remotely.

By the end of this lab, you'll understand how AgentCore Gateway enables tool reusability and scalability. The same tools you built locally now become production-ready APIs that any authorized agent can access, with built-in authentication, monitoring, and automatic scaling.

<div class="alert-warning">
    <b>Warning:</b>
    <p>
    1) please make sure to run <b>00-prerequisites.ipynb</b> to properly setup all the packages.
    </p>
    <p>
    2) please make sure to run <b>create-an-sports-agent.ipynb</b> to setup knowledge base and dynamodb resources.
    </p>
</div>

## Initialize Environment

In [ ]:
import boto3
import json
import sys
import os
import time
import sagemaker
from pathlib import Path
from IPython.display import display, Markdown

# Import display helpers
sys.path.insert(0, str(Path.cwd().parent / 'helper'))
from display_helper import print_success, print_info, print_result

# Setup session
boto_session = boto3.session.Session()
sess = sagemaker.Session(boto_session=boto_session)
region = sess.boto_region_name
account_id = boto3.client('sts').get_caller_identity()['Account']

print_info(f"Region: {region}")
print_info(f"Account ID: {account_id}")

## Configuration

Restore resources from Lab 1 and configure naming for new resources.

**What This Does:**
- Restores `kb_id`, `dynamo_table`, and `bda_result` from Lab 1 using Jupyter's `%store` magic
- Creates unique resource names using region and account ID prefix
- Ensures no naming conflicts across AWS accounts

**Resources Configured:**
- Lambda function name for hosting tools
- IAM role name for Lambda execution permissions

In [ ]:
%store -r lab_kb_id
%store -r lab_dynamo_table
%store -r bda_result

In [ ]:
# From previous notebook
prefix = 'sports-analysis-agent'
suffix = f"{region}-{account_id[:3]}"

# Lambda configuration
lab_lambda_function_name = f'sports-tools-lambda-{suffix}'
lab_lambda_role_name = f'sports-tools-lambda-role-{suffix}'

print_info(f"Lambda Function: {lab_lambda_function_name}")
print_info(f"DynamoDB Table: {lab_dynamo_table}")
print_info(f"Knowledge Base ID: {lab_kb_id}")

## Import Helper Functions

Import custom helper classes that simplify AWS service interactions.

**Helper Classes:**
- **LambdaHelper** - Creates Lambda functions with IAM roles and policies
- **CognitoHelper** - Manages Cognito User Pools, resource servers, and OAuth clients
- **AgentCoreHelper** - Creates AgentCore Gateway resources and IAM roles

These helpers abstract complex AWS API calls into simple methods.

In [ ]:
sys.path.insert(0, '..')

from helper.lambda_helper import LambdaHelper
# Initialize Lambda helper
lambda_helper = LambdaHelper()

from helper.cognito_helper import CognitoHelper

# Initialize Cognito helper
cognito_helper = CognitoHelper()

from helper.agentcore_helper import AgentCoreHelper

# Initialize AgentCore helper
agentcore_helper = AgentCoreHelper()

## Lambda Function Package

Verify the pre-packaged Lambda deployment package exists.

**Package Contents:**
- **lambda_function.py** - Python handler with two tools:
  - `retrieve_match_info` - Queries Bedrock Knowledge Base for match details
  - `lookup_player_info` - Queries DynamoDB for player information
- **api_spec.json** - MCP tool specifications defining:
  - Tool names and descriptions
  - Input parameters and types
  - Output schemas

**Lambda Deployment Package:**
- Pre-built zip file with all dependencies
- Includes boto3 and required libraries
- Ready for deployment to AWS Lambda

**Reference:** [Lambda Deployment Packages](https://docs.aws.amazon.com/lambda/latest/dg/python-package.html)

In [ ]:
# Verify the pre-packaged Lambda zip file exists
lambda_zip_path = 'lambda/sports_tools_lambda.zip'

if os.path.exists(lambda_zip_path):
    file_size = os.path.getsize(lambda_zip_path)
    print_success(f"Lambda package found: {lambda_zip_path} ({file_size:,} bytes)")
else:
    print(f"❌ Lambda package not found at {lambda_zip_path}")

## Create IAM Role for Lambda

Create an IAM execution role that grants Lambda permissions to access AWS services.

**IAM Role Components:**

1. **Trust Policy** - Allows Lambda service to assume this role
2. **Managed Policies** - AWS-managed policies for basic Lambda execution:
   - `AWSLambdaBasicExecutionRole` - CloudWatch Logs permissions
3. **Custom Policies** - Additional permissions for:
   - **DynamoDB** - `GetItem`, `Query` on player table
   - **Bedrock** - `Retrieve` on Knowledge Base

**Why This Matters:**
- Lambda functions run with no permissions by default
- IAM role grants least-privilege access to only required resources
- Follows AWS security best practices

**Reference:** [Lambda Execution Role](https://docs.aws.amazon.com/lambda/latest/dg/lambda-intro-execution-role.html)

In [ ]:
# Define additional IAM policies for DynamoDB and Bedrock
additional_policies = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "dynamodb:GetItem",
                "dynamodb:Query"
            ],
            "Resource": f"arn:aws:dynamodb:{region}:{account_id}:table/{lab_dynamo_table}"
        },
        {
            "Effect": "Allow",
            "Action": ["bedrock:Retrieve"],
            "Resource": f"arn:aws:bedrock:{region}:{account_id}:knowledge-base/{lab_kb_id}"
        }
    ]
}

# Create IAM role using lambda_helper
role_arn = lambda_helper.create_lambda_role(
    role_name=lab_lambda_role_name,
    additional_policies=additional_policies
)

time.sleep(20)

print_success(f"IAM role created: {lab_lambda_role_name}")
print_info(f"Role ARN: {role_arn}")

## Create Lambda Function

Deploy the Lambda function with the sports tools.

**Lambda Configuration:**

- **Runtime:** Python 3.12
- **Handler:** `lambda_function.lambda_handler` - Entry point for function execution
- **Timeout:** 60 seconds - Maximum execution time
- **Memory:** 256 MB - Allocated memory (also determines CPU allocation)
- **Environment Variables:**
  - `SPORTS_KB_ID` - Knowledge Base ID for match information retrieval
  - `DYNAMODB_TABLE` - Table name for player lookups

**How Lambda Works:**
1. Lambda receives event with tool name and parameters
2. Function routes to appropriate tool handler
3. Tool executes (queries DynamoDB or Knowledge Base)
4. Returns result in standardized format

**Reference:** [Lambda Functions](https://docs.aws.amazon.com/lambda/latest/dg/welcome.html)

In [ ]:
# Define environment variables
environment_variables = {
    'SPORTS_KB_ID': lab_kb_id,
    'DYNAMODB_TABLE': lab_dynamo_table
}

# Create Lambda function using lambda_helper
result = lambda_helper.create_gateway_lambda(
    lambda_function_code_path=lambda_zip_path,
    function_name=lab_lambda_function_name,
    role_name=lab_lambda_role_name,
    additional_policies=additional_policies,
    environment_variables=environment_variables,
    timeout=60,
    memory_size=256
)

if result['exit_code'] == 0:
    lambda_arn = result['lambda_function_arn']
    print_success(f"Lambda function created: {lab_lambda_function_name}")
    print_info(f"ARN: {lambda_arn}")
else:
    print(f"❌ Error: {result['lambda_function_arn']}")

## Test Lambda Function

Invoke the Lambda function directly to verify it works before connecting to the gateway.

**Test Process:**

1. **Wait for Lambda to be Active** - New Lambda functions start in `Pending` state
   - Checks function state every 5 seconds
   - Proceeds when state is `Active`

2. **Invoke Lambda Synchronously** - Uses `RequestResponse` invocation type
   - Sends test event with tool name and parameters
   - Waits for response before continuing

3. **Verify Response** - Checks that:
   - Lambda executes without errors
   - DynamoDB query returns player data
   - Response format matches expected schema

**Test Event Structure:**
```json
{
  "tool_name": "lookup_player_info",
  "parameters": {
    "team_name": "Alpine Wolves",
    "player_number": "8"
  }
}
```

**Reference:** [Lambda Invocation](https://docs.aws.amazon.com/lambda/latest/dg/lambda-invocation.html)

In [ ]:
# Test player lookup
lambda_client=boto3.client('lambda')

print("Waiting for Lambda to be ready...")
while True:
    state = lambda_client.get_function(FunctionName=lab_lambda_function_name)['Configuration']['State']
    if state == 'Active':
        print("✅ Lambda is ready")
        break
    print(f"Lambda state: {state}, waiting...")
    time.sleep(5)

test_event = {
    'tool_name': 'lookup_player_info',
    'parameters': {
        'team_name': 'Alpine Wolves',
        'player_number': '8'
    }
}

response = lambda_client.invoke(
    FunctionName=lab_lambda_function_name,
    InvocationType='RequestResponse',
    Payload=json.dumps(test_event)
)

result = json.loads(response['Payload'].read())
print("Player Lookup Test:")
print(json.dumps(result, indent=2))

## Setup Cognito for Gateway Authentication

Create Amazon Cognito resources to provide JWT-based authentication for the AgentCore Gateway.

**What is Amazon Cognito?**
- Managed identity service for user authentication and authorization
- Supports OAuth 2.0 and OpenID Connect standards
- Issues JWT tokens for secure API access

**Components Created:**

1. **User Pool** - Identity provider that manages authentication
   - Acts as OAuth 2.0 authorization server
   - Issues access tokens and refresh tokens
   - Provides OIDC discovery endpoint

2. **Resource Server** - Defines custom OAuth scopes
   - `gateway:read` - Read access to gateway tools
   - `gateway:write` - Write access to gateway tools
   - Scopes control fine-grained permissions

3. **App Client (M2M)** - Machine-to-machine authentication
   - Uses client credentials flow (no user interaction)
   - Receives client ID and client secret
   - Exchanges credentials for access token

**Authentication Flow:**
```
1. Agent requests token from Cognito (client_id + client_secret)
2. Cognito validates credentials and returns JWT access token
3. Agent includes token in Authorization header
4. Gateway validates token against Cognito discovery URL
5. If valid, gateway allows tool invocation
```

**Reference:** 
- [Cognito User Pools](https://docs.aws.amazon.com/cognito/latest/developerguide/cognito-user-identity-pools.html)
- [OAuth 2.0 Client Credentials](https://docs.aws.amazon.com/cognito/latest/developerguide/token-endpoint.html)

In [ ]:
# Configuration
USER_POOL_NAME = f"sports-gateway-pool-{suffix}"
RESOURCE_SERVER_ID = "sports-gateway-id"
RESOURCE_SERVER_NAME = "sports-gateway-server"
CLIENT_NAME = f"sports-gateway-client-{suffix}"

SCOPES = [
    {"ScopeName": "gateway:read", "ScopeDescription": "Read access"},
    {"ScopeName": "gateway:write", "ScopeDescription": "Write access"}
]

lab_scope_string = f"{RESOURCE_SERVER_ID}/gateway:read {RESOURCE_SERVER_ID}/gateway:write"

print(f"User Pool Name: {USER_POOL_NAME}")
print(f"Client Name: {CLIENT_NAME}")

In [ ]:
# Create or retrieve Cognito resources
print("Creating or retrieving Cognito resources...")

lab_user_pool_id = cognito_helper.get_or_create_user_pool(USER_POOL_NAME)
print(f"✅ User Pool ID: {lab_user_pool_id}")

cognito_helper.get_or_create_resource_server(
    user_pool_id=lab_user_pool_id,
    resource_server_id=RESOURCE_SERVER_ID,
    resource_server_name=RESOURCE_SERVER_NAME,
    scopes=SCOPES
)
print("✅ Resource server ensured")

lab_client_id, lab_client_secret = cognito_helper.get_or_create_m2m_client(
    user_pool_id=lab_user_pool_id,
    client_name=CLIENT_NAME,
    resource_server_id=RESOURCE_SERVER_ID
)
print(f"✅ Client ID: {lab_client_id}")

# Get discovery URL
lab_cognito_discovery_url = f'https://cognito-idp.{region}.amazonaws.com/{lab_user_pool_id}/.well-known/openid-configuration'
print(f"✅ Discovery URL: {lab_cognito_discovery_url}")

## Create IAM Role for AgentCore Gateway

Create an IAM role that allows the AgentCore Gateway to invoke Lambda functions.

**Gateway IAM Role:**

- **Trust Policy** - Allows `bedrock-agentcore.amazonaws.com` service to assume this role
- **Permissions** - Grants `lambda:InvokeFunction` on the sports tools Lambda
- **Purpose** - Gateway uses this role to call Lambda when tools are invoked

**Why This is Needed:**
- Gateway runs as an AWS service and needs permissions to invoke Lambda
- Follows AWS security model of service-to-service authentication
- Enables least-privilege access (only specific Lambda function)

**Reference:** [IAM Roles for Services](https://docs.aws.amazon.com/IAM/latest/UserGuide/id_roles_create_for-service.html)

In [ ]:
# Create gateway IAM role
lab_gateway_name = f'sports-gateway-lab2-{suffix}'
agentcore_gateway_iam_role = agentcore_helper.create_agentcore_gateway_role(lab_gateway_name)

lab_gateway_role_arn = agentcore_gateway_iam_role['Role']['Arn']
print(f"✅ Gateway IAM Role ARN: {lab_gateway_role_arn}")

## Create AgentCore Gateway

Create the Amazon Bedrock AgentCore Gateway that exposes Lambda tools via MCP protocol.

**What is AgentCore Gateway?**
- Managed service that exposes backend services as agent tools
- Implements Model Context Protocol (MCP) for tool invocation
- Provides authentication, authorization, and request routing
- Enables tool sharing across multiple agents

**Gateway Configuration:**

- **Protocol Type:** `MCP` - Model Context Protocol for agent-tool communication
- **Authorizer Type:** `CUSTOM_JWT` - Uses Cognito JWT tokens for authentication
- **Authorizer Configuration:**
  - `allowedClients` - List of Cognito client IDs that can access gateway
  - `discoveryUrl` - Cognito OIDC discovery endpoint for token validation
- **Role ARN:** IAM role for invoking Lambda functions

**How Gateway Works:**
```
1. Agent sends MCP request with JWT token
2. Gateway validates token against Cognito
3. Gateway checks client ID is in allowedClients list
4. Gateway routes request to appropriate Lambda target
5. Lambda executes and returns result
6. Gateway forwards response to agent
```

**Gateway States:**
- `CREATING` - Gateway is being provisioned
- `READY` - Gateway is ready to accept requests
- `FAILED` - Gateway creation failed

**Reference:** [AgentCore Gateway](https://docs.aws.amazon.com/bedrock/latest/userguide/agentcore-gateway.html)

In [ ]:
gateway_client = boto3.client('bedrock-agentcore-control', region_name=region)

# Configure Cognito authorizer
auth_config = {
    "customJWTAuthorizer": {
        "allowedClients": [lab_client_id],
        "discoveryUrl": lab_cognito_discovery_url
    }
}

# Create gateway
create_response = gateway_client.create_gateway(
    name=lab_gateway_name,
    roleArn=lab_gateway_role_arn,
    protocolType='MCP',
    authorizerType='CUSTOM_JWT',
    authorizerConfiguration=auth_config,
    description='AgentCore Gateway for sports tools'
)

lab_gateway_id = create_response["gatewayId"]
lab_gateway_url = create_response["gatewayUrl"]

print(f"✅ Gateway created successfully!")
print(f"   Gateway ID: {lab_gateway_id}")
print(f"   Gateway URL: {lab_gateway_url}")

## Add Lambda as Gateway Target

Configure the Lambda function as a gateway target, making the sports tools available via MCP.

**What is a Gateway Target?**
- Backend service (Lambda, API, etc.) that gateway can invoke
- Defines how gateway communicates with the service
- Includes tool schema for MCP protocol

**Target Configuration:**

1. **Wait for Gateway Ready** - Gateway must be in `READY` state before adding targets

2. **Load API Spec** - `api_spec.json` defines MCP tool schema:
   ```json
   {
     "name": "lookup_player_info",
     "description": "Look up player information",
     "inputSchema": { "team_name": "string", "player_number": "string" }
   }
   ```

3. **Target Configuration** - Specifies:
   - **Lambda ARN** - Which Lambda function to invoke
   - **Tool Schema** - MCP tool definitions from api_spec.json
   - **Protocol** - MCP for agent communication

4. **Credential Configuration** - How gateway authenticates to Lambda:
   - `GATEWAY_IAM_ROLE` - Uses gateway's IAM role to invoke Lambda
   - Alternative: `ASSUME_ROLE` for cross-account access

**Tool Naming Convention:**
- Gateway prefixes tool names with target name
- Format: `{target_name}___{tool_name}`
- Example: `sports-tools-target___lookup_player_info`

**Reference:** [Gateway Targets](https://docs.aws.amazon.com/bedrock/latest/userguide/agentcore-gateway-targets.html)

In [ ]:
# Load the API spec for the tools
with open('lambda/api_spec.json', 'r') as f:
    api_spec = json.load(f)

print("Loaded API spec with tools:")
for tool in api_spec:
    print(f"  - {tool['name']}: {tool['description']}")

In [ ]:
# Wait for gateway to be ready
print("Waiting for gateway to be ready...")
while True:
    status = gateway_client.get_gateway(gatewayIdentifier=lab_gateway_id)['status']
    if status == 'READY':
        print("✅ Gateway is ready")
        break
    print(f"Gateway status: {status}, waiting...")
    time.sleep(5)

In [ ]:
# Configure Lambda target
lambda_target_config = {
    "mcp": {
        "lambda": {
            "lambdaArn": lambda_arn,
            "toolSchema": {
                "inlinePayload": api_spec
            }
        }
    }
}

# Configure credentials
credential_config = [
    {
        "credentialProviderType": "GATEWAY_IAM_ROLE"
    }
]

# Create gateway target
lab_target_name = f'sports-tools-target-{suffix}'
response = gateway_client.create_gateway_target(
    gatewayIdentifier=lab_gateway_id,
    name=lab_target_name,
    description='Lambda target for sports tools',
    targetConfiguration=lambda_target_config,
    credentialProviderConfigurations=credential_config
)

lab_target_id = response['targetId']
print(f"✅ Gateway target created successfully!")
print(f"   Target ID: {lab_target_id}")
print(f"   Target Name: {lab_target_name}")

## Test Gateway with Agent

Create an agent that connects to the gateway and uses the remote tools to analyze the sports video.

**Integration Steps:**

1. **Get Access Token** - Request JWT token from Cognito
   - Uses client credentials flow (client_id + client_secret)
   - Token valid for 1 hour (configurable in Cognito)
   - Token includes granted scopes (gateway:read, gateway:write)

2. **Create MCP Client** - Connect to gateway using Strands MCP client
   - `streamablehttp_client` - HTTP transport for MCP protocol
   - Includes Authorization header with Bearer token
   - Gateway URL from gateway creation response

3. **List Available Tools** - Query gateway for available tools
   - Gateway returns tool definitions from api_spec.json
   - Tools include names, descriptions, and input schemas
   - Agent uses this to understand what tools it can call

4. **Create Agent** - Initialize Strands agent with:
   - **Model** - Claude Sonnet for reasoning and tool selection
   - **Tools** - Remote tools from gateway
   - **System Prompt** - Instructions for sports video analysis

5. **Invoke Agent** - Send query with video metadata
   - Agent analyzes video summary
   - Determines which tools to call
   - Invokes tools through gateway
   - Synthesizes final answer

**MCP Protocol Flow:**
```
Agent → MCP Client → Gateway (validates JWT) → Lambda → AWS Services
                                                    ↓
Agent ← MCP Client ← Gateway ← Lambda ← Results
```

**Reference:** 
- [MCP Protocol](https://modelcontextprotocol.io/)
- [Strands Agents](https://strandsagents.com/)

In [ ]:
# Get access token
token_response = cognito_helper.get_token(
    user_pool_id=lab_user_pool_id,
    client_id=lab_client_id,
    client_secret=lab_client_secret,
    scope_string=lab_scope_string
)

token = token_response["access_token"]
print("Token response:", token)

In [ ]:
from strands.models import BedrockModel
from mcp.client.streamable_http import streamablehttp_client 
from strands.tools.mcp.mcp_client import MCPClient
from strands import Agent

def create_streamable_http_transport():
    return streamablehttp_client(lab_gateway_url,headers={"Authorization": f"Bearer {token}"})

client = MCPClient(create_streamable_http_transport)

agent_model_id = 'global.anthropic.claude-sonnet-4-5-20250929-v1:0'

In [ ]:
SYSTEM_PROMPT = """You are a sports video analysis assistant that answers user queries about the provided sports video.

You have video metadata, as well as additional tools to get more information from match reports and player table.

Base on the information you obtained from metadata and tools, generate a clear and accurate answer to the user query. DO NOT answer queries beyond the game you are reviewing.
"""

model = BedrockModel(
    model_id=agent_model_id,
    temperature=0.3,
)

In [ ]:
query = "who are the players in the video?"

In [ ]:
PROMPT_TEMPLATE="""
VIDEO Metadata: {video_summaries}

USER QUERY: {query}
"""

prompt = PROMPT_TEMPLATE.replace("{video_summaries}", bda_result["video"]["summary"])
prompt = prompt.replace("{query}", query)

In [ ]:
with client:
    # Call the listTools 
    tools = client.list_tools_sync()
    # Create an Agent with the model and tools
    agent = Agent(model=model,tools=tools) ## you can replace with any model you like
    print(f"Tools loaded in the agent are {agent.tool_names}")
    # result = client.call_tool_sync(
    #     tool_use_id="get-order-id-123-call-1", # You can replace this with unique identifier. 
    #     name=target_name+"___lookup_player_info", # This is the tool name based on AWS Lambda target types. This will change based on the target name
    #     arguments={
    #             'team_name': 'Alpine Wolves',
    #             'player_number': '8'
    #         }
    #     )
    # # Print the MCP Tool response
    # print(f"Tool Call result: {result['content'][0]['text']}")

    result = agent(prompt=prompt)

    # Parse result
    print("\nParsing agent response...")
    try:
        response_text = result.get('output', result) if isinstance(result, dict) else str(result)
        print_result("Final Agent Response",response_text)

    except Exception as e:
        print(f"Error parsing response: {e}")
        print(f"error : {str(e)},\nraw_response: {str(result)}")

In [ ]:
# Store variables for next notebook
%store lab_gateway_id
%store lab_gateway_url
%store lab_client_id
%store lab_client_secret
%store lab_user_pool_id
%store lab_scope_string
%store lab_cognito_discovery_url
%store lab_lambda_function_name
%store lab_lambda_role_name
%store lab_gateway_name

print("✅ Variables stored for next notebook")

## 🎉 Gateway Setup Complete!

You've successfully setup remote MCP tools using AgentCore Gateway!